# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Техническое задание:
Метрика качества модели *F1* не меньше 0.75. 

**План выполнения проекта**

1. Загрузить и подготовьть данные.
2. Обучить разные модели. 
3. Сделайть выводы.

Для выполнения проекта будет применяться класс TfidfVectorizer().

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходимые инструменты для работы.

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.utils import shuffle
pd.set_option('max_colwidth', 100)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import spacy

Загрузим данные и посмотрим общую информацию.

In [3]:
df = pd.read_csv(r'C:\Users\Admin\text_recognition\toxic_comments.csv')#, index_col = [0])

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Проверим на пропуски и дубликаты, уникальные значения в столбце toxic.

In [6]:
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df['toxic'].unique()

array([0, 1], dtype=int64)

Лемматизируем текст коментариев.

In [9]:
nlp = spacy.load("en_core_web_sm")

In [10]:
def lem(row): 
    doc = nlp(row['text'])
    doc = " ".join([token.lemma_ for token in doc])
    return doc
df['lemm_t'] = df.apply(lem, axis=1)

In [11]:
df.head()

,Unnamed: 0,text,toxic,lemm_t
0,0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,Explanation \n why the edit make under my username Hardcore Metallica Fan be revert ? they be no...
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,"D'aww ! he match this background colour I be seemingly stick with . thank . ( talk ) 21:51 , J..."
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,"hey man , I be really not try to edit war . it be just that this guy be constantly remove releva..."
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,""" \n More \n I can not make any real suggestion on improvement - I wonder if the section statist..."
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0,"you , sir , be my hero . any chance you remember what page that be on ?"


In [12]:
def clear_text(row):
    text=row['lemm_t']
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.split()
    text = " ".join(text)
    return text
df['lemm_text'] = df.apply(clear_text, axis=1)

In [13]:
#!{sys.executable} -m spacy download en

In [14]:
#df.to_csv('df_l_cl.csv')

Посмотрим на датасет который получился.

In [15]:
df.head(3)

,Unnamed: 0,text,toxic,lemm_t,lemm_text
0,0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,Explanation \n why the edit make under my username Hardcore Metallica Fan be revert ? they be no...,Explanation why the edit make under my username Hardcore Metallica Fan be revert they be not van...
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,"D'aww ! he match this background colour I be seemingly stick with . thank . ( talk ) 21:51 , J...",D aww he match this background colour I be seemingly stick with thank talk January UTC
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,"hey man , I be really not try to edit war . it be just that this guy be constantly remove releva...",hey man I be really not try to edit war it be just that this guy be constantly remove relevant i...


Удалим ненужные столбцы

In [16]:
df = df.drop(['Unnamed: 0', 'text', 'lemm_t'], axis = 1)

In [17]:
df.head()

,toxic,lemm_text
0,0,Explanation why the edit make under my username Hardcore Metallica Fan be revert they be not van...
1,0,D aww he match this background colour I be seemingly stick with thank talk January UTC
2,0,hey man I be really not try to edit war it be just that this guy be constantly remove relevant i...
3,0,More I can not make any real suggestion on improvement I wonder if the section statistic should ...
4,0,you sir be my hero any chance you remember what page that be on


Теперь в таблице остался только лиматизированный сортированный текст и орветы проиндексированные по порядку.

Подготовим данные для обучения моделей.

In [18]:
features = df.drop(['toxic'], axis = 1)
target = df['toxic']
X_train, X_TV, y_train, y_TV = \
train_test_split(features, target, test_size = 0.4, random_state=0)
X_valid, X_test, y_valid, y_test = \
train_test_split(X_TV, y_TV, test_size = 0.5, random_state=0)
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords)) 


In [19]:
train_tf = count_tf_idf.fit_transform(X_train['lemm_text'].values)
valid_tf = count_tf_idf.transform(X_valid['lemm_text'].values)
test_tf = count_tf_idf.transform(X_test['lemm_text'].values)
print(train_tf.shape)
print(valid_tf.shape)
print(test_tf.shape)

(95575, 115865)
(31858, 115865)
(31859, 115865)


Вывод:
Столбец таблицы с постами был поготовлен(лемитизирован и очищеен), преобразован в векторы.


## Обучение

Обучаем логистическую регрессию, 

Для выбора наилучшей модели будем баланс  класса,  остальные гиперпараметры по умолчанию т.к. при выборе  модели с подбором гиперпараметров требует неприемливо много времяни. Думаю что основная проблема в решение этой задачи найти баланс недообучения
и переобучения модели, чем больше данных тем больше вероятность переобучить модель, чем меньше данных тем больше вероятность 
недообучить модель. Будем эксперементировать с тренировочной выборкой.

#### Логистическая регрессия.

In [20]:
model_l = LogisticRegression(random_state=12345, class_weight='balanced')
model_l.fit(train_tf, y_train)
pred = pd.DataFrame(model_l.predict(valid_tf))
print('f1 мера логистической регресии на валидатационной выборке равна: ',f1_score(y_valid, pred)) 

f1 мера логистической регресии на валидатационной выборке равна:  0.7595762021189895


#### Случайный лес.

In [ ]:
model_fc = RandomForestClassifier(random_state=12345, class_weight='balanced')
model_fc.fit(train_tf, y_train)
pred_fc = model_fc.predict(valid_tf) 
print('f1 мера случайного леса на валидатационной выборке равна: ', f1_score(pred_fc, y_valid)) 

#### Дерево решений

In [ ]:
model_d = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model_d.fit(train_tf, y_train)
pred_d = model_d.predict(valid_tf)
print('f1 мера случайного леса на валидатационной выборке равна: ', f1_score(pred_d, y_valid))

#### CatBoostClassifier

In [ ]:
model_cat =  CatBoostClassifier(verbose=False, random_state=12345)
model_cat.fit(train_tf, y_train)
pred_cat = model_cat.predict(valid_tf)
print('f1 мера CatBoostClassifier леса на валидатационной выборке равна: ', f1_score(pred_cat, y_valid))

Лучший результат покзаля модель CatBoost, почти f1 = 0.766.

### Применим функцию upsample

Теперь попробуем изменить количество положительного класса. По всей видимости
модель получается недообученной. Исправим это.
   Напишем функцию которая будет добавлять в датасет положительный класс.
Перебирать с помощью цикла не будем т.к. в ходе эксрериментов лучший параметр
repeat для каждой модели уже был установлен, а вычисление путем объядинение всех
моделей в цикл для подбора repeat потребует неприемлемо много времяни, будем 
использовать, что уже извесно.

In [ ]:
def upsample(features, target, repeat, count_tf_idf):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    features_upsampled = count_tf_idf.fit_transform(features_upsampled['lemm_text'].values)
    return features_upsampled, target_upsampled

In [ ]:
features_upsampled, target_upsampled = upsample(X_train, y_train, 7, count_tf_idf)

#### Модель случайный лес

In [ ]:
model_fc = RandomForestClassifier(random_state=12345)
model_fc.fit(features_upsampled, target_upsampled)
predicted_valid = model_fc.predict(valid_tf)
f1_fc = f1_score(y_valid, predicted_valid)
print("F1 случайного леса на валидционной выборке равен: ", f1_fc)

####  Модель дерево решений

In [ ]:
model_d = DecisionTreeClassifier(random_state=12345, max_depth = 500)#500 деревьев т.к. входе экспериментов было установленно что после 70 деревьев метрика начинает колебаться от 0.7 до 0.74
model_d.fit(features_upsampled, target_upsampled)
p_d = model_d.predict(valid_tf)
f1_d = f1_score(y_valid, p_d)
print('F1 дерево решений на валидционной выборке равен: ', f1_d)

#### Модель логистическая регрессия

In [ ]:
model_l = LogisticRegression(random_state=12345)
model_l.fit(features_upsampled, target_upsampled)
p_l = model_l.predict(valid_tf)
f1_l = f1_score(y_valid, p_l)
print('f1 мера логистической регрессии на валидатационной выборке равна: ', f1_l)

#### Модель градиентного спуска 

In [ ]:
features_upsampled, target_upsampled = upsample(X_train, y_train, 3, count_tf_idf)#параметр repeat поставим равный 3 т.к. в ходе экспериментов было установленно при repeat=3 лучший результат 

In [ ]:
model_c = CatBoostClassifier(verbose=False, random_state=12345)
model_c.fit(features_upsampled, target_upsampled)
p = model_c.predict(valid_tf)
pred = pd.DataFrame(p)
pred.columns = ['toxic']
f1_score(y_valid, pred)

In [ ]:
models = [LogisticRegression(random_state=12345), CatBoostClassifier(verbose=False, random_state=12345)]
#DecisionTreeClassifier(random_state=12345),#RandomForestClassifier(random_state=12345)],
f1_list = []
model_list = []
repeat = []

In [ ]:
def f1_scor(features_upsampled, target_upsampled, valid_tf, model):
    model.fit(features_upsampled, target_upsampled)
    return f1_score(y_valid, model.predict(valid_tf)), model

In [ ]:
for i in range(4,7):
    for model in models:    
        features_upsampled, target_upsampled = upsample(X_train, y_train, i, count_tf_idf)
        f1, model = f1_scor(features_upsampled, target_upsampled, valid_tf, model)
        f1_list.append(f1)
        model_list.append(model)
        repeat.append(i)
    print(i-3,'-я  Итерация')

In [ ]:
data = {'f1 мера': f1_list,'модель': model_list, 'коэфициент увеличения полож. класса': repeat}
data = pd.DataFrame(data = data)

In [ ]:
data

   Из-за  объемности данных пришлось разбить исследование на несколько частей  т.к. ядро 
не выдерживает, в общую функцию весь анализ поместить не удалось. 
   Самую высокую f1 = 0.787 меру показала модель  CatBoostClassifier с параметром функции
repeat = 3.

Попробуем изменить подход к тренировке модели. Сформируем тренировочную выборку из положительных
ответов и части отрицательных с изменением количества отрицательных. Может быть удасться добиться
баланса между переобучением и недообучением модели.

In [ ]:
def shap(features, target):
    x, x_t, y, y_t = train_test_split(features, target, test_size = 0.2, random_state=12345)
    stopwords = set(nltk_stopwords.words('english'))
    tf_idf = TfidfVectorizer(stop_words=list(stopwords)) 
    train_tf = tf_idf.fit_transform(x['lemm_text'].values)
    
    test_tf = tf_idf.transform(x_t['lemm_text'].values)
    
    model = CatBoostClassifier(verbose=False, random_state=12345)
    model.fit(train_tf, y)
    pr = f1_score(y_t, model.predict(test_tf))
    print()
    print(pr)
    print()
    return pr, model, tf_idf

In [ ]:
li = [];ln = [];l = [];p_v = []
for i in range(12,32):
    df_new_1 = X_train[y_train==1]
    df_new_0 = X_train[y_train==0]
    n = round(df_new_1.shape[0]*i/10)
    df_new_0 = df_new_0.sample(n = n, random_state = 12345)
    df_new = pd.concat([df_new_0]+[df_new_1])
    pr, model_c1, tf_idf  = shap(df_new, y_train[df_new.index])
    li.append(i/10)
    ln.append(n)
    l.append(pr)
    val_tf = tf_idf.transform(X_valid['lemm_text'].values)
    tes_tf = tf_idf.transform(X_test['lemm_text'].values)
    print(val_tf.shape)
    print(tes_tf.shape)
    pv = model_c1.predict(val_tf)
    p_val = f1_score(y_valid, pv)
    p_v.append(p_val)
    print(p_val)


In [ ]:
d = {'часть от положительных':ln,'колличество отрицательных ответов':li, 'f1':l, 'f1 мера на валидатационной выборке':p_v}
lm = pd.DataFrame(data = d)
lm

Из таблицы видно что самая высокая f1 мера при соотношении в тренировочной
выборки 3-х частей отрицательных и одной с положительными ответами.


Вывод:
   В ходе поиска наилучшей модели были выполнены действия, были проаналезированны
четыре модели - логистическая регрессия, дерево решений, случайный лес и  CatBoost.
Первоначально модели были проанализированны без трансформации тренировочной выборки,
что дало низкий результат. По техническому заданию f1 мера должна привысить 0.75.
При первичном анализе результаты на валидационной выборке получились слижком низкие,
логистическая регрессия и CatBoost дали результат близкий к техническому заданию, но на
валидатационной выборке, что совсем не гарантирует выполнения его на тестовой выборке,
а деревянные модели показали вообше низкий результат.  В дальнейшем я добавил в гипер
параметры class_weight='balanced'(взвешивания классов), это дало прирост метрики около
2-5 процентов на моделях первых трех моделях. В дальнейшем  помощью техники upsampling
я изменил размер обучающей выборки, этот метод дал наилучший  результат f1 = 0.7877 на
валидационной выборке модели CatBoost.
   В ходе дальнейшего исследования был применен метод по трансформации тренировочной 
выборки, что значительно уменьшило время обучения.


## Тестирование модели

Для тестирования выберем модель CatBoost которая получилась с помощью применения
техники upsampling.  

In [ ]:
p = model_c.predict(test_tf)
f1 = f1_score(y_test, p)
print('f1 мера модели CatBoost регрессии на тестовой выборке равна: ', f1)

## Выводы

#### Общий вывод:
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Была найдена и обучена модель классифицировать комментарии на позитивные и негативные. 
Модель определяет таксичность поста, метрика F1 = 0.778  что соответствует тех. заданию.

